In [20]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import scipy.sparse as sp

In [12]:
def normalized_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    normalized_adj = normalized_adj.astype(np.float32)
    return normalized_adj
    
def sparse_to_tuple(mx):
    mx = mx.tocoo()
    coords = np.vstack((mx.row, mx.col)).transpose()
    L = tf.SparseTensor(coords, mx.data, mx.shape)
    return tf.sparse_reorder(L) 
    
def calculate_laplacian(adj, lambda_max=1):  
    adj = normalized_adj(adj + sp.eye(adj.shape[0]))
    adj = sp.csr_matrix(adj)
    adj = adj.astype(np.float32)
    return sparse_to_tuple(adj)

def weight_variable_glorot(input_dim, output_dim, name=""):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.random_uniform([input_dim, output_dim], minval=-init_range,
                            maxval=init_range, dtype=tf.float32)

    return tf.Variable(initial,name=name)  

In [30]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
      
    trainX1 = np.array(trainX)
    trainY1 = np.array(trainY)
    testX1 = np.array(testX)
    testY1 = np.array(testY)
    return trainX1, trainY1, testX1, testY1

In [13]:
class GCN(object):
    def __init__(self, num_units, adj, inputs, output_dim, activation=tf.nn.tanh,
                input_size = None, num_proj = None, reuse = None, **kwargs):
        super(GCN, self).__init__(**kwargs)
        
        self._num_units = num_units
        self._output_dim = output_dim
        self._inputs = inputs
        self._num_nodes = inputs.get_shape()[2].value
        self._input_dim = inputs.get_shape()[1].value
        self._batch_size = tf.shape(inputs)[0]
        self._adj = []
        self._adj.append(calculate_laplacian(adj))
        self._activation = activation
        self._gconv()
        
    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    @property
    def output_size(self):
        output_size = self._num_units
        return output_size
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
           
    def _gconv(self,scope=None):
        inputs = self._inputs       
        inputs = tf.transpose(inputs, perm=[2,0,1])
        x0 = tf.reshape(inputs,shape=[self._num_nodes,self._batch_size*self._input_dim])
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope):
            for adj in self._adj:
                x1 = tf.sparse_tensor_dense_matmul(adj, x0)
            x1 = tf.reshape(x1,shape=[self._num_nodes,self._batch_size,self._input_dim])
            x1 = tf.reshape(x1,shape=[self._num_nodes*self._batch_size,self._input_dim])
            
            weights = weight_variable_glorot(self._input_dim, self.output_size, name='weights')
            self.hidden1 = self._activation(tf.matmul(x1, weights))
                                 
            weights = weight_variable_glorot(self.output_size,self._output_dim, name='weights')
            self.output = tf.matmul(self.hidden1, weights)
            self.output = tf.reshape(self.output,shape=[self._num_nodes,self._batch_size,
                                                        self._output_dim])
            self.output = tf.transpose(self.output, perm=[1,2,0])
            self.output = tf.reshape(self.output,shape=[-1,self._num_nodes])

In [14]:
mpath = '../Data/Gangnam/'

In [15]:
data_path = mpath + 'speed_gangnam.csv'
adj_path = mpath + 'adj_mx_gangnam.pkl'

In [19]:
data = pd.read_csv(data_path)
data = data.drop('Unnamed: 0', axis=1)
data

,1210007700,1210008500,1210009500,1210010300,1210011300,1210013700,1220011900,1220016300,1220021100,1220025100,...,1210013800,1210013000,1210012200,1210012300,1210013100,1210013900,1210014900,1210015900,1210017100,1210018300
0,32.70,21.00,33.54,43.21,24.31,26.54,35.91,29.00,28.78,21.76,...,24.98,24.54,32.76,43.75,32.36,32.13,35.47,22.96,26.62,14.00
1,31.50,38.00,25.61,43.89,22.75,24.37,30.73,29.07,25.92,22.66,...,26.22,25.14,32.15,43.75,25.00,35.04,26.00,16.00,27.22,29.58
2,22.00,38.00,47.00,54.00,19.00,24.36,35.75,24.63,21.96,22.67,...,21.55,25.16,34.37,43.75,25.00,33.00,36.32,22.42,32.40,11.00
3,31.00,34.39,32.70,44.15,23.64,27.76,40.06,41.00,28.65,21.68,...,25.33,24.11,33.41,28.61,29.88,34.93,37.35,21.35,29.07,32.34
4,39.20,37.65,30.75,43.66,22.76,28.04,28.10,23.25,23.00,19.92,...,24.44,24.15,34.94,28.11,33.88,33.53,39.30,30.68,27.68,30.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,27.50,36.33,29.53,39.86,26.15,20.05,25.37,18.27,21.88,18.25,...,28.17,24.28,38.21,21.95,29.47,28.47,31.52,32.13,28.22,15.96
2876,25.51,37.62,30.48,41.88,26.52,23.00,31.94,16.56,24.29,19.14,...,28.65,24.49,32.52,27.44,31.76,33.00,44.68,24.89,33.04,11.00
2877,29.40,28.26,31.81,39.68,26.45,24.28,28.92,18.00,23.41,21.11,...,32.26,23.88,32.02,26.94,30.75,31.10,17.00,24.70,30.21,18.65
2878,24.00,28.21,29.02,41.93,25.49,25.38,25.57,17.61,23.63,21.70,...,27.08,27.32,34.00,26.27,27.32,29.02,37.90,27.58,30.23,15.41


In [26]:
with open(adj_path, 'rb') as file:
    temp = pickle.load(file)
adj = temp[2]
adj.shape, adj

((506, 506),
 array([[1.        , 0.40327677, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.40327677, 1.        , 0.91133416, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.91133416, 1.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 1.        , 0.38495174,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.38495174, 1.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ]], dtype=float32))

In [59]:
time_len = data.shape[0]
num_nodes = data.shape[1]
time_len, num_nodes

(2880, 506)

In [28]:
train_rate = 0.8
seq_len = 12
pre_len = 3

In [31]:
data1 = np.mat(data, dtype=np.float32)

max_value = np.max(data1)
data1 /= max_value
trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len)

In [32]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((2289, 12, 506), (2289, 3, 506), (561, 12, 506), (561, 3, 506))

In [33]:
batch_size = 32
totalbatch = int(trainX.shape[0] / batch_size)
training_data_count =len(trainX)

In [34]:
totalbatch, training_data_count

(71, 2289)

In [ ]:
self._adj = []
        self._adj.append(calculate_laplacian(adj))
        self._activation = activation
        self._gconv()
        
    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    @property
    def output_size(self):
        output_size = self._num_units
        return output_size
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
           
    def _gconv(self,scope=None):
        inputs = self._inputs       
        inputs = tf.transpose(inputs, perm=[2,0,1])
        x0 = tf.reshape(inputs,shape=[self._num_nodes,self._batch_size*self._input_dim])
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope):
            for adj in self._adj:
                x1 = tf.sparse_tensor_dense_matmul(adj, x0)
            x1 = tf.reshape(x1,shape=[self._num_nodes,self._batch_size,self._input_dim])
            x1 = tf.reshape(x1,shape=[self._num_nodes*self._batch_size,self._input_dim])
            
            weights = weight_variable_glorot(self._input_dim, self.output_size, name='weights')
            self.hidden1 = self._activation(tf.matmul(x1, weights))
                                 
            weights = weight_variable_glorot(self.output_size,self._output_dim, name='weights')
            self.output = tf.matmul(self.hidden1, weights)
            self.output = tf.reshape(self.output,shape=[self._num_nodes,self._batch_size,
                                                        self._output_dim])
            self.output = tf.transpose(self.output, perm=[1,2,0])
            self.output = tf.reshape(self.output,shape=[-1,self._num_nodes])

In [60]:
adjs = []
adjs.append(calculate_laplacian(adj))
adjs

In [63]:
data.values.shape

(2880, 506)